## Electromagnetic Analysis for FPGA_Tro_Ez

In [69]:
from scipy.io import loadmat


In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
sns.set()

In [71]:
%matplotlib


Using matplotlib backend: Qt5Agg


In [72]:
fpga_tro_ez = loadmat('FreqEzFPGAtro.mat')

In [73]:
fpga_tro_ez.keys()


dict_keys(['__header__', '__version__', '__globals__', 'FreqMatrixExtractHY'])

In [74]:
data = fpga_tro_ez['FreqMatrixExtractHY']

In [75]:
x = np.arange(0,160,1)
y = np.arange(0, 160, 1)


In [76]:
X, Y = np.meshgrid(x,y)

In [18]:
for i in range(10):
    plt.subplots()
    clabel = plt.contourf(X, Y, data[i], cmap=plt.cm.RdBu_r)
    plt.colorbar(clabel)

C:\Anaconda3\lib\site-packages\matplotlib\pyplot.py:528: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [ ]:
for i in range(len(data)):
    plt.subplots()
    clabel = plt.contourf(X, Y, data[i], cmap=plt.cm.RdYlBu_r)
    plt.colorbar(clabel)
    plt.savefig("./Figures2/Fig" + str(i+1))
    plt.close()

In [81]:
freq_std = []
for i in range(len(data)):
    freq_std.append(data[i].std())

In [82]:
freq_std = pd.DataFrame(freq_std)

In [83]:
freq_std.plot()

In [18]:
freq_std[freq_std>=3.3].dropna(how='all')

,0
40,3.572578
42,4.535333
124,3.576768
127,3.524094
128,3.459161
208,3.692222
211,3.688005
212,3.618184
213,3.398591
218,4.882679


In [77]:
std_x = np.arange(0,1001,1)

In [78]:
len(std_x)

1001

In [79]:
plt.subplots()
plt.bar(std_x,freq_std.values.flatten(),width=0.5)

NameError: name 'freq_std' is not defined

In [29]:
freq_std.shape

(1001, 1)

In [33]:
len(freq_std.values.flatten())

1001

In [45]:
x_ticks = np.arange(200,1400,200)

In [47]:
plt.xticks(x_ticks)

([<matplotlib.axis.XTick at 0x1f8e8358>,
 <a list of 6 Text xticklabel objects>)

In [50]:
plt.subplots()

(<matplotlib.figure.Figure at 0x221cc048>,
 <matplotlib.axes._subplots.AxesSubplot at 0x223092e8>)

In [49]:
sns.set()

In [51]:
plt.scatter(freq_std,freq_std)

##### 1.PCA学习

（1）PCA analysis

In [84]:
from sklearn.decomposition import PCA

In [53]:
data0 = data[0]

In [54]:
data0.shape

(160, 160)

In [55]:
pca = PCA(n_components=5)

In [56]:
pca.fit(data0)

PCA(copy=True, iterated_power='auto', n_components=5, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [57]:
pca.n_components

5

In [59]:
pca.components_.shape

(5, 160)

In [61]:
rng = np.random.RandomState(1)

In [62]:
X = np.dot(rng.rand(2,2),rng.randn(2,200)).T

In [63]:
X.shape

(200, 2)

In [65]:
plt.subplots()


(<matplotlib.figure.Figure at 0x223181d0>,
 <matplotlib.axes._subplots.AxesSubplot at 0x2345e358>)

In [66]:
plt.scatter(X[:,0],X[:,1])

In [67]:
pca = PCA(2)

In [68]:
pca.fit(X)

PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

主成分:

In [69]:
pca.components_

array([[-0.94446029, -0.32862557],
       [-0.32862557,  0.94446029]])

可解释的差异：

In [71]:
pca.explained_variance_

array([0.7625315, 0.0184779])

In [72]:
def draw_vector(v0, v1, ax=None):
    ax = ax or plt.gca()
    arrowprops = dict(arrowstyle='->',linewidth=2, shrinkA=0, shrinkB=0)
    ax.annotate('', v0, v1, arrowprops=arrowprops)

In [73]:
plt.subplots()

(<matplotlib.figure.Figure at 0x23b3d0f0>,
 <matplotlib.axes._subplots.AxesSubplot at 0x23b3d710>)

In [74]:
plt.scatter(X[:,0], X[:,1], alpha=0.2)

In [75]:
for length, vector in zip(pca.explained_variance_, pca.components_):
    v = vector * 3 * np.sqrt(length)
    draw_vector(pca.mean_, pca.mean_ + v)
plt.axis('equal')

(-2.7370333024013895,
 2.5780365361094546,
 -0.9431003378113761,
 1.0148960105226839)

下面做个PCA降维的例子，数据还是使用先前的X：

In [77]:
pca = PCA(n_components=1)#一个主成分

In [78]:
pca.fit(X)

PCA(copy=True, iterated_power='auto', n_components=1, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [80]:
X_pca = pca.transform(X)

In [81]:
X_pca.shape

(200, 1)

In [83]:
print("Original shape:", X.shape)
print('Transformed shape:', X_pca.shape)

Original shape: (200, 2)
Transformed shape: (200, 1)


变换的数据被投影到单一维度。为了理解降维的效果，可对数据做降维的逆变换，并且与原数据仪器画出来。

In [84]:
X_new = pca.inverse_transform(X_pca)#降维的逆变换
#画图
plt.subplots()
plt.scatter(X[:,0], X[:,1], alpha=0.2) #alpha表示透明度，值越小越透明
plt.scatter(X_new[:,0], X_new[:,1], alpha=0.8)
plt.axis('equal')


(-2.760354895836614, 2.650583711737073, -0.987217466350432, 1.0169344322370828)

图中浅色的点是原始数据点，深色的点是降维后的数据，可以看出，沿着非主轴的信息被舍弃了，只留下最大方差的数据成分。被舍弃的那一部分信息，可以认为是数据降维后损失的信息。

（2）PCA用于手写数字识别的可视化

In [85]:
from sklearn.datasets import load_digits  #导入手写数据
digits = load_digits()
digits.data.shape

(1797, 64)

由于这些数字都是以图片的形式，像素8*8存储的，因此它是64维的。我们将它投影到2维看看效果如何。

In [87]:
pca = PCA(n_components=2)
projected = pca.fit_transform(digits.data)#fit()方法和transform()方面合并在一起了
print(digits.data.shape)
print(projected.shape)

(1797, 64)
(1797, 2)


然后画出每个点的前两个主成分，以更好地了解数据。

In [88]:
plt.subplots()
plt.scatter(projected[:,0], projected[:,1], c=digits.target, edgecolors='none', alpha=0.5, 
            cmap=plt.cm.get_cmap('spectral', 10))
plt.xlabel('component 1')
plt.ylabel('component 2')
plt.colorbar()


C:\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: The spectral and spectral_r colormap was deprecated in version 2.0. Use nipy_spectral and nipy_spectral_r instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


如何选择主成分的数量呢？可以使用“累计方差贡献率”来确定主成分的数量。

In [89]:
pca = PCA().fit(digits.data)
plt.subplots()
plt.plot(np.cumsum(pca.explained_variance_ratio_)) #可解释方差贡献率的累计和
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')


Text(0,0.5,'cumulative explained variance')

这个曲线量化了在前N个主成分中包含多少总的64维的方差。从图中可以看出，前10个主成分基本上包含了75%左右的方差。
当选择到50个成分时，基本就包含了100%的方差。
从图中我们也知道，上述2维的PCA损失大部分的信息，要保持90%以上的方差，至少选择20个主成分。

（3）将PCA用于噪声过滤

通常情况下，任何成分的方差都远大于噪声的方差。因此，如果只用主成分去重构数据（比如图像等），那么应该能够选择性去除噪声的。
首先，我们画出几个无噪声的手写数字。

In [92]:
def plot_digits(data):
    fig, axes = plt.subplots(4, 10, figsize=(10,4), subplot_kw={'xticks':[],'yticks':[]},
                            gridspec_kw=dict(hspace=0.1, wspace=0.1))
    for i, ax in enumerate(axes.flat):
        ax.imshow(data[i].reshape(8,8), cmap='binary', interpolation='nearest', clim=(0, 16))
plot_digits(digits.data)

给数据添加噪声，重新画图。

In [108]:
np.random.seed(42)
noisy_data = np.random.normal(digits.data, 4)
plot_digits(noisy_data)

然后用噪声数据训练PCA，投影后保留50%的方差。

In [109]:
pca = PCA(0.50).fit(noisy_data)
pca.n_components_

12

可以看出，保留的50%的方差对应12个主成分，然后利用逆变换重构手写数字。

In [110]:
components = pca.transform(noisy_data)
filtered_data = pca.inverse_transform(components)
plot_digits(filtered_data)

(4)PCA用于辅助k-means进行聚类分析

In [111]:
data.shape

(1001, 160, 160)

In [112]:
data0.shape

(160, 160)

In [125]:
pca = PCA().fit(data[43])
plt.subplots()
plt.plot(np.cumsum(pca.explained_variance_ratio_)) #可解释方差贡献率的累计和
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.title('FPGA_Tro_Ez: variance ratio of frequency')

Text(0.5,1,'FPGA_Tro_Ez: variance ratio of frequency')

In [123]:
pca = PCA(0.999999).fit(data[43])
pca.n_components_

80

<font size=3>数据集中的方差验证</font>

In [134]:
d0_flat = data[0].flatten()
d0_mean = d0_flat.mean()
print(d0_mean)
sum(d0_flat) / len(d0_flat)

s = 0
for i in d0_flat:
    s = s + (i - d0_mean)**2

d0_std = np.sqrt(s/len(d0_flat))
print(d0_std)
print(d0_flat.std())
    

-42.746578730705465
1.2621245419701919
1.262124541970186


In [168]:
for i in range(15):
    print(1+i,data[i].std())


1 1.2621245419701863
2 3.1853907810845747
3 1.0737758660333292
4 3.165710803915547
5 1.0192067022425138
6 3.2278245356025956
7 1.0993517537151964
8 3.180998063116527
9 1.0575961715900501
10 3.163043733504071
11 1.0170475021968557
12 3.2111906817310523
13 1.1277354610516404
14 3.2144007820347786
15 1.011385910667333


In [170]:
print(data[41].std())
print(data[42].std())

3.220667595865842
4.535332530024808


##### 2.K-Means学习

<font size=3>k-means算法在不带标签的多维数据中寻找确定数量的簇，它要符合以下两个假设：<br/>
（1）簇的中心点是属于该簇中各坐标点的算数平均值；<br/>
（2）簇中各点到该簇中心点比到其它簇的中心点的距离最短。
</font>

In [10]:
from sklearn.datasets.samples_generator import make_blobs
X, y_true = make_blobs(n_samples=300, centers=4, cluster_std=0.6, random_state=0)
plt.subplots()
plt.scatter(X[:,0], X[:,1], s=50)

<img src="5-11-1.png" title="图5-11-1">

可以看出，图中可分为4个簇。下面使用k-means算法来进行聚类。

In [11]:
from sklearn.cluster import KMeans
kmean = KMeans(n_clusters=4)
kmean.fit(X)
y_means = kmean.predict(X)
print("每个点的聚类结果:\n",y_means)
print("\n簇的中心点:\n",kmean.cluster_centers_)

每个点的聚类结果:
 [3 1 0 1 3 3 2 0 1 1 2 1 0 1 3 0 0 3 2 2 3 3 0 2 2 0 3 0 2 0 1 1 0 1 1 1 1
 1 2 3 0 2 0 0 2 2 1 2 1 3 2 3 1 3 3 2 1 2 1 3 1 0 1 2 2 2 1 3 1 2 0 2 1 2
 2 1 2 0 3 1 3 0 3 3 1 0 3 0 1 1 0 3 1 2 2 0 3 3 0 2 1 3 1 3 0 3 3 0 1 0 2
 2 3 1 3 0 1 3 3 0 2 3 2 3 3 3 3 2 3 2 1 2 2 3 1 2 2 1 0 1 1 2 0 2 0 2 1 0
 1 1 1 0 1 0 3 2 1 2 3 0 1 0 0 3 0 2 2 0 3 0 0 1 3 0 2 1 3 3 0 2 3 0 2 2 0
 0 0 0 3 1 0 2 0 0 2 2 2 0 2 1 0 2 3 2 0 1 2 1 0 1 0 2 0 0 1 2 2 3 3 0 1 3
 3 2 3 2 0 1 1 0 0 1 0 3 2 0 3 2 1 2 3 0 3 1 1 1 1 2 2 1 0 2 3 0 2 2 2 3 3
 1 0 0 2 3 1 2 0 1 0 3 3 2 2 0 3 3 3 0 1 1 3 3 0 3 3 3 1 2 1 0 3 3 1 1 1 3
 3 0 1 2]

簇的中心点:
 [[ 0.94973532  4.41906906]
 [-1.37324398  7.75368871]
 [-1.58438467  2.83081263]
 [ 1.98258281  0.86771314]]


我们将原始数据分别以不同的颜色画出来，然后画出K-means算法算出的簇中心点。

In [14]:
plt.subplots()
plt.scatter(X[:,0], X[:,1], c=y_means, s=50, cmap="viridis")#画出原始数据
centers = kmean.cluster_centers_ #获取簇中心点
plt.scatter(centers[:,0], centers[:,1], c='black', s = 200, alpha=0.6)#画出簇中心点


<img src="5-11-2.png" title="图5-11-2">

k-means算法是如何快速地找到簇中心点的呢，毕竟簇中心点的分配组合方案会随着数据的增加而呈现指数级的增长，如果k-means算法逐个穷举这些组合方案将会耗费大量的时间。好在k-means可以采用***<font >期望最大化<font>***（expectation maximization,E-M）规避穷举。下面简单介绍期望最大化算法。E-M算法的步骤：<br/>
（1）猜测一些簇中心点。<br/>
（2）重复一下步骤直至收敛：<br/>
 &emsp;&emsp; a.期望步骤（E-step）：将点分配距离其最近的中心点<br/>
 &emsp;&emsp; b.最大化步骤（M-step）：将簇中心点设置为该簇中所有点坐标的平均值<br/>
    
下面简单实现k-means算法:


In [12]:
from sklearn.metrics import pairwise_distances_argmin
def find_cluster(X, n_clusters, rseed=1):
    '''
    描述：对X进行聚类。
    参数：
        X：array-like，通常是(n_samples, n_features)的数组
        n_clusters:int类型，要聚成几类
        rseed：int类型，用于设置随机种子
    返回值：
        centers：所有簇的中心点
        labels：X中每个点所属的那个簇的索引
    
    '''
    #随机选择簇中心
    rng = np.random.RandomState(rseed)
    i = rng.permutation(X.shape[0])[:n_clusters] #permutation()用于随机搅乱np.arange(X.shape[0])
    centers = X[i]
    
    while True:
        #对于X中的每一行数据构成的点，从centers中找到距离该点最近的点
        labels = pairwise_distances_argmin(X, centers)
        #print("labels's shape:", labels.shape)
        #print(labels)
        #更新簇中心：计算每一个簇中所有点的平均值，更新该簇的中心点
        new_centers = np.array([X[labels == j].mean(axis=0) for j in range(n_clusters)])
        #print(X[labels == i])
        #检查是否收敛
        if np.all(centers == new_centers):
            break
        centers = new_centers
        
    return centers,labels


       

In [42]:
n_clusters = 6
X, y_true = make_blobs(n_samples=300, centers=n_clusters, cluster_std=0.6, random_state=0)
#自己实现的k-means算法
centers, labels = find_cluster(X,n_clusters,2)
plt.subplots()
plt.scatter(X[:,0], X[:,1], c=labels, s=50, cmap='viridis')
plt.scatter(centers[:,0], centers[:,1], c='red', s=380, alpha=0.6)
plt.title('find_cluster()')

kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit_predict(X)
plt.subplots()
plt.scatter(X[:,0], X[:,1], c=labels, s=50, cmap='viridis')
plt.scatter(centers[:,0], centers[:,1], c='red', s=380, alpha=0.6)
plt.title('KMeans class:')

Text(0.5,1,'KMeans class:')

上述用find_cluster()方法实现的k-means算法对于不同的簇中心初始值可能不会收敛到全局最优解，因此需要尝试多种不同的簇中心初始值才有可能达到全局最优解。在sklearn.clusters中的KMeans构造函数中，可以通过设置n_init参数选择需要尝试多少次簇中心初始值，默认是10。<br/>
**注意**：k-means算法的缺点是必须认为地指定簇的数量算法才能够去很好聚类。

#### k-means算法只能确定线性聚类边界

k-means的基本模型假设是：与其它簇的中心相比，数据点更加接近自己的簇中心点，因此，如果簇中心点呈现非线性的复杂形状时，k-means算法经常不起作用。比如：

In [13]:
from sklearn.datasets import  make_moons
X, y = make_moons(n_samples=200, noise=0.05, random_state=0) #noise是要添加到数据的高斯噪声标准差
labels = KMeans(n_clusters=2, random_state=0).fit_predict(X)
plt.subplots()
plt.scatter(X[:,0], X[:,1], c=labels, s=50, cmap='viridis')


<img src="5-11-3.png">

解决的方法是通过核变换将数据投影到更高维的空间，从而使线性分离成为可能。这样就可以让k-means算法也可以出来非线性的边界问题。<br/>
这种核k-means算法在sklearn的SpectralCulstering评估器中实现，它使用最近邻图来计算数据的高维表示，然后用k-means算法分配标签进行聚类。如下：


In [14]:
from sklearn.cluster  import SpectralClustering
model = SpectralClustering(n_clusters=2, affinity='nearest_neighbors', assign_labels="kmeans")
labels = model.fit_predict(X)
plt.subplots()
plt.scatter(X[:,0], X[:,1], c=labels, s=50, cmap='viridis')

C:\Anaconda3\lib\site-packages\sklearn\manifold\spectral_embedding_.py:234: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


<img src="5-11-4.png">

从上图可以看出，通过核变换，k-means就可以适应非线性边界的情况了。

由于k-means算法每次更新簇中心都要获取所有数据，当数据量比较大，k-means会很慢。解决的方法是“每次迭代更新只使用数据集的一个子集来更新簇中心”这就是批处理的k-means算法的核心思想。该算法在sklearn.cluster.MiniBatchKMeans中实现，该算法的接口和标准的KMeans接口相同。

### 用k-means识别手写数字

In [15]:
from sklearn.datasets import load_digits
from sklearn.cluster import KMeans
digits = load_digits()
print(digits.data.shape)

(1797, 64)


In [16]:
kmeans = KMeans(n_clusters=10, random_state=0)
clusters = kmeans.fit_predict(digits.data)
print(clusters.shape)
print(kmeans.cluster_centers_.shape)#簇中心点
print(type(kmeans.cluster_centers_))

(1797,)
(10, 64)
<class 'numpy.ndarray'>


In [27]:
fig, ax = plt.subplots(2, 5, figsize=(8, 3))
centers = kmeans.cluster_centers_.reshape(10, 8, 8)
for axi, center in zip(ax.flat, centers):
    axi.set(xticks=[], yticks=[])
    axi.imshow(center, interpolation='nearest', cmap=plt.cm.binary)

<img src='5-11-5.png'>

从图中可以发现，即使没有标签，k-means也能找到可辨识的数字中心。但是1和8例外。同时由于k-means算法不知道簇的真实标签，因此0~9的数字不是顺序排列的。我们可以将学到的簇标签与真实的标签进行匹配，从而解决这个问题。

In [18]:
from scipy.stats import  mode
labels = np.zeros_like(clusters)
for i in range(10):
    mask = (clusters == i)
    labels[mask] = mode(digits.target[mask])[0]#mode用来计算众数

然后使用sklearn.metrics的accuracy_score()方法评估k-means识别手写数字的准确性。

In [19]:
from sklearn.metrics import accuracy_score
print(accuracy_score(digits.target, labels))

0.7935447968836951


画出混淆矩阵，直观地查看算法的不足之处。

In [20]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
sns.set()

mat = confusion_matrix(digits.target, labels)
plt.subplots()
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False, xticklabels=digits.target_names,
            yticklabels=digits.target_names)
plt.xlabel("True Label")
plt.ylabel("Predicted Label")

Text(115.922,0.5,'Predicted Label')

<img src='5-11-6.png'>

可以看出，混淆的地方主要是1和8，5也有轻量的混淆。可以看出，在不需要标签的情况下，k-means算法介意构成一个数字分类器。此外，我们可以引入t-SNE算法（t-分布邻域嵌入算法），该算法是一个非线性嵌入算法，十分适合保留簇中的数据点。

In [21]:
from sklearn.manifold import TSNE
#投影数据
tsne = TSNE(n_components=2, init='pca', random_state=0)#n_components:嵌入的空间的维度；init：嵌入空间的初始化
digits_projection = tsne.fit_transform(digits.data)

#再使用k-means算法聚类
kmeans = KMeans(n_clusters=10, random_state=0)
clusters = kmeans.fit_predict(digits_projection)

#再排列标签
labels = np.zeros_like(clusters)
for i in range(10):
    mask = (clusters == i)
    labels[mask] = mode(digits.target[mask])[0]

#计算准确度
print(accuracy_score(digits.target, labels))


0.9371174179187535


可以看出，同样在没有标签的情况下，此时的准确率达到了94%。

### 3.电磁场数据分析与验证

In [86]:
fpga_hy_data_n9030a = loadmat('FPGA/N9030_1Hy.mat')

In [87]:
fpga_hy_data_n9030a.keys()

dict_keys(['__header__', '__version__', '__globals__', 'YStrucTMP'])

In [88]:
hy_n9030a = fpga_hy_data_n9030a['YStrucTMP']

In [89]:
print(hy_n9030a.shape)
print(hy_n9030a[0].sum())


(25600, 1001)
-101145.53464781001


In [90]:
hy_n9030a = pd.DataFrame(hy_n9030a)
cols = []
for i in hy_n9030a.columns.values:
    cols.append('f' + str(i+1))
hy_n9030a.columns =cols
print(hy_n9030a.columns.values)



['f1' 'f2' 'f3' ... 'f999' 'f1000' 'f1001']


In [91]:
hy_std_n9030a = []
for i in hy_n9030a.columns.values:
    hy_std_n9030a.append(hy_n9030a[i].std())

hy_std_n9030a = pd.DataFrame(hy_std_n9030a)
hy_std_n9030a.plot()
plt.title('unshuffled data of N9030A')    

Text(0.5,1,'unshuffled data of N9030A')

In [92]:
fpga_hy_data = loadmat('FPGA/FreqHYFPGAtro.mat')

In [93]:
fpga_hy_data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'FreqMatrixExtractHY'])

In [94]:
hy = fpga_hy_data['FreqMatrixExtractHY']


In [95]:
print(hy.shape)
print(hy[0].sum())

(1001, 160, 160)
-1040467.4403513799


In [96]:
hy_std = []
for i in range(len(hy)):
    hy_std.append(hy[i].std())

hy_std = pd.DataFrame(hy_std)
hy_std.plot()
plt.title('shuffled data of N9030A')

Text(0.5,1,'shuffled data of N9030A')

In [97]:
FreqMatrixExtract = loadmat('FPGA/FreqEzFPGAtro.mat')



In [98]:
print(FreqMatrixExtract.keys())
FreqMatrixExtract = FreqMatrixExtract['FreqMatrixExtractHY']

dict_keys(['__header__', '__version__', '__globals__', 'FreqMatrixExtractHY'])


In [99]:
print(FreqMatrixExtract.shape)
freq1_data = FreqMatrixExtract[0]
#print(freq1_data)
freq1_data.std(axis=0).std()

(1001, 160, 160)


0.0237759175676074

In [208]:
FreqMatrixExtract[0][0,:][:5]

array([-43.89454902, -43.89454902, -44.339358  , -44.339358  ,
       -43.99094365])

In [106]:
std_x = np.linspace(0,1200,1001)
print(std_x)


[   0.     1.2    2.4 ... 1197.6 1198.8 1200. ]


In [123]:
freq1_data_std = []
for i in range(len(FreqMatrixExtract)):
    freq1_data_std.append(FreqMatrixExtract[i].std(axis=0).std())#先对行求std，所得的结果再求std
#print(freq1_data_std)
freq1_data_std = pd.DataFrame(freq1_data_std)


freq1_data_std.index = std_x
#print(freq1_data_std)
freq1_data_std.plot()

plt.subplots()
plt.bar(std_x,freq1_data_std.values.flatten())


<Container object of 1001 artists>

In [124]:
print(freq1_data_std.values.flatten())

[0.02377592 0.21905074 0.08994633 ... 0.05702146 0.25777524 0.53199141]


In [174]:
freq1_data_std

,0
0.0,0.023776
1.2,0.219051
2.4,0.089946
3.6,0.195654
4.8,0.061209
6.0,0.232395
7.2,0.107463
8.4,0.226238
9.6,0.077044
10.8,0.207003


<img src="5-11-7.png">

（1）对N9030_1Ez的原始数据进行重排验证

In [138]:
ez_fpga = loadmat('N9030_1Ez.mat')

In [141]:
ez_fpga_data = ez_fpga['ZStrucTMP']

In [142]:
ez_fpga_data[:,0]

array([-41.74544771, -41.49575077, -41.49226662, ..., -39.21595956,
       -39.16485879, -39.20550713])

In [210]:
type(ez_fpga_data)


numpy.ndarray

In [184]:
ind = np.arange(160,25600,160)


In [185]:
ez_fpga_data[:,0]

array([-41.74544771, -41.49575077, -41.49226662, ..., -39.21595956,
       -39.16485879, -39.20550713])

In [234]:
ez_arr = np.zeros_like(FreqMatrixExtract)
for col in range(ez_fpga_data.shape[1]):
    arr_list = np.hsplit(ez_fpga_data[:,col],ind)
    for i in range(len(arr_list)):
        if i % 2 != 0:
            arr_list[i] = arr_list[i][::-1]
    ez_arr[col] = np.array(arr_list)


In [239]:
#print(ez_arr.shape)
#print(ez_arr[0])
std_x = np.linspace(0,1200,1001)
ez_arr_std = []
for i in range(len(ez_arr)):
    ez_arr_std.append(ez_arr[i].std(axis=0).std())#先对行求std，所得的结果再求std
#print(freq1_data_std)
ez_arr_std = pd.DataFrame(ez_arr_std)


ez_arr_std.index = std_x
#print(freq1_data_std)
ez_arr_std.plot()
#print(ez_arr_std[:11],'\n\n')
plt.subplots()
plt.bar(std_x,ez_arr_std.values.ravel())
#print(ez_arr_std.values.ravel()[:10],'\n\n')

<Container object of 1001 artists>

In [241]:
len(ez_arr_std[ez_arr_std>=0.5].dropna())

8

In [200]:
for i in range(len(arr_list)):
    
    if i % 2 != 0:
        arr_list[i] = arr_list[i][::-1]
arr_list[0][-5:]        

array([-41.51549425, -41.51433286, -41.79074158, -41.76286843,
       -41.55614259])

In [201]:
arr_df = pd.DataFrame(arr_list)
arr_df.shape

(160, 160)

In [202]:
arr_df.std(axis=1).std()

0.018437944980989173